Задача 2

    2.1 Условие
Вам доступны следующие фреймы данных:
А)Таблица с информацией о среднедневном спросе на каждый товар для каждой локации:

product	location	demand
1	01	100
1	02	110
2	01	120
2	02	90
3	01	70
3	02	80

product — уникальный идентификатор продукта

location — уникальный идентификатор локации

demand — значение среднедневного спроса на конкретный товар в конкретной локации в единицах товара

Б)Таблица с информацией о складских запасах на уровне месяца:

product	location	stock
1	01	1000
1	02	400
2	01	300
2	02	250

product — уникальный идентификатор продукта
stock — уровень запасов в единицах товара на уровне месяца


    2.2 Задание

В качестве примера рассмотрим июнь 2023 года. 
Вам необходимо сформировать витрину данных, в которой для каждой пары товар-локация на уровне каждой технической недели* будет рассчитано прогнозируемое значение количества проданных товаров (с учетом среднедневного спроса) и количество остатков товара на складе. 

Примечание:
*Техническая неделя — это календарная неделя или часть календарной недели, которая «укладывается» в один календарный месяц. 
Например, в августе 2023 года вам доступны следующие технические недели: 
01.08—06.08
07.08—13.08
14.08—20.08
21.08—27.08
28.08—31.08

In [1]:
# Импортирование необходимых библиотек pyspark
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
# Инициализация Spark-сессии
spark = SparkSession.builder \
    .master("local") \
    .appName("onlineretail") \
    .getOrCreate()

24/08/17 14:52:40 WARN Utils: Your hostname, alto-pc resolves to a loopback address: 127.0.1.1; using 192.168.181.138 instead (on interface eth0)
24/08/17 14:52:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/17 14:52:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/17 14:52:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Таблица с информацией о среднедневном спросе на каждый товар для каждой локации
# Создание DataFrame

data2a = ((1, "01", 100), \
    (1, "02", 110), \
    (2, "01", 120), \
    (2, "02", 90), \
    (3, "01", 70), \
    (3, "02", 80), \
  )

columns2a = ["productA", "locationA", "demand"]

sparkDF2a = spark.createDataFrame(data = data2a, schema = columns2a)

sparkDF2a.printSchema()
sparkDF2a.show(truncate=False)

root
 |-- productA: long (nullable = true)
 |-- locationA: string (nullable = true)
 |-- demand: long (nullable = true)



+--------+---------+------+
|productA|locationA|demand|
+--------+---------+------+
|1       |01       |100   |
|1       |02       |110   |
|2       |01       |120   |
|2       |02       |90    |
|3       |01       |70    |
|3       |02       |80    |
+--------+---------+------+



In [4]:
# Таблица с информацией о складских запасах на уровне месяца:
# Создание DataFrame

data2b = ((1, "01", 1000), \
    (1, "02", 400), \
    (2, "01", 300), \
    (2, "02", 250), \
  )

columns2b = ["productB", "locationB", "stock"]

sparkDF2b = spark.createDataFrame(data = data2b, schema = columns2b)

sparkDF2b.printSchema()
sparkDF2b.show(truncate=False)

root
 |-- productB: long (nullable = true)
 |-- locationB: string (nullable = true)
 |-- stock: long (nullable = true)

+--------+---------+-----+
|productB|locationB|stock|
+--------+---------+-----+
|1       |01       |1000 |
|1       |02       |400  |
|2       |01       |300  |
|2       |02       |250  |
+--------+---------+-----+



Необходимо сформировать витрину данных, в которой для каждой пары товар-локация на уровне каждой технической недели* будет рассчитано прогнозируемое значение количества проданных товаров (с учетом среднедневного спроса) и количество остатков товара на складе.

In [5]:
# Таблица с информацией о технических неделях июня 2023 года:
# Создание DataFrame

t_weeks = ((1, 1, 4, 4), \
    (2, 5, 11, 7), \
    (3, 12, 18, 7), \
    (4, 19, 25, 7), \
    (5, 26, 30, 5), \
  )
columns2b = ["week_number", "start_day", "end_day", "days_count"]
spark_t_weeks = spark.createDataFrame(data = t_weeks, schema = columns2b)

In [6]:
showcase = sparkDF2a.join(sparkDF2b\
               , on = ((sparkDF2a.productA == sparkDF2b.productB) \
               & (sparkDF2a.locationA == sparkDF2b.locationB))\
                , how = "full").select(F.col("productA").alias("product"),\
                                       F.col("locationA").alias("location"),\
                                       F.col("demand"),
                                       F.col("stock"))
showcase = showcase.na.fill(0)                                        
showcase.show()

+-------+--------+------+-----+
|product|location|demand|stock|
+-------+--------+------+-----+
|      1|      01|   100| 1000|
|      1|      02|   110|  400|
|      2|      01|   120|  300|
|      2|      02|    90|  250|
|      3|      01|    70|    0|
|      3|      02|    80|    0|
+-------+--------+------+-----+



In [7]:
# Витрина данных, в которой для каждой пары товар-локация 
# на уровне каждой технической недели рассчитано 
# прогнозируемое значение количества проданных товаров (sold_week_)
# (с учетом среднедневного спроса) 
# и количество остатков товара на складе (stock_week_).

showcase_weeks = showcase\
    .withColumn("sold_week1", F.when(F.lit(F.col("demand")*4) < F.col("stock"),\
        F.lit(F.col("demand")*4))\
        .otherwise(F.col("stock")))\
    .withColumn("stock_week1",\
        F.when((F.lit(F.col("stock") - F.col("demand")*4)) > 0,\
        F.lit(F.col("stock") - F.col("demand")*4))\
        .otherwise(0))\
    .withColumn("sold_week2", F.when(F.lit(F.col("demand")*7) < F.col("stock_week1"),\
        F.lit(F.col("demand")*7))\
        .otherwise(F.col("stock_week1")))\
    .withColumn("stock_week2", \
        F.when((F.lit(F.col("stock_week1") - F.col("demand")*(7))) > 0,\
        F.lit(F.col("stock_week1") - F.col("demand")*(7)))\
        .otherwise(0))\
    .withColumn("sold_week3", F.when(F.lit(F.col("demand")*7) < F.col("stock_week2"),\
        F.lit(F.col("demand")*7))\
        .otherwise(F.col("stock_week2")))\
    .withColumn("stock_week3", \
        F.when((F.lit(F.col("stock") - F.col("demand")*(4+7+7))) > 0,\
        F.lit(F.col("stock") - F.col("demand")*(4+7+7)))\
        .otherwise(0))\
    .withColumn("sold_week4", F.when(F.lit(F.col("demand")*7) < F.col("stock_week3"),\
        F.lit(F.col("demand")*7))\
        .otherwise(F.col("stock_week3")))\
    .withColumn("stock_week4", \
        F.when((F.lit(F.col("stock") - F.col("demand")*(4+7+7+7))) > 0,\
        F.lit(F.col("stock") - F.col("demand")*(4+7+7+7)))\
        .otherwise(0))\
    .withColumn("sold_week5", F.when(F.lit(F.col("demand")*5) < F.col("stock_week4"),\
        F.lit(F.col("demand")*5))\
        .otherwise(F.col("stock_week4")))\
    .withColumn("stock_week5", \
        F.when((F.lit(F.col("stock") - F.col("demand")*(4+7+7+7+5))) > 0,\
        F.lit(F.col("stock") - F.col("demand")*(4+7+7+7+5)))\
        .otherwise(0))

showcase_weeks.show()

+-------+--------+------+-----+----------+-----------+----------+-----------+----------+-----------+----------+-----------+----------+-----------+
|product|location|demand|stock|sold_week1|stock_week1|sold_week2|stock_week2|sold_week3|stock_week3|sold_week4|stock_week4|sold_week5|stock_week5|
+-------+--------+------+-----+----------+-----------+----------+-----------+----------+-----------+----------+-----------+----------+-----------+
|      1|      01|   100| 1000|       400|        600|       600|          0|         0|          0|         0|          0|         0|          0|
|      1|      02|   110|  400|       400|          0|         0|          0|         0|          0|         0|          0|         0|          0|
|      2|      01|   120|  300|       300|          0|         0|          0|         0|          0|         0|          0|         0|          0|
|      2|      02|    90|  250|       250|          0|         0|          0|         0|          0|         0|       

24/08/17 14:52:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
